In [ ]:
#| default_exp data_preprocessing

# Data Preprocessing

> oh no


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import zarr, numpy as np, pandas as pd, multiprocessing as mp, torch
from functools import partial

In [ ]:
#| export
def calculate_class_weights(dataloader, ignore_index=-100, returns_padded_mask=True, return_ratio=True):
    if returns_padded_mask:
        target_values = [y.unique(return_counts=True) for _,y,_ in dataloader]
    else:
        target_values = [y.unique(return_counts=True) for _,y in dataloader]
    all_counts = []
    for i,c in target_values:
        if ignore_index is not None:
            all_counts.append(c[i != ignore_index])
        else:
            all_counts.append(c)
    idx = i[i != ignore_index] if ignore_index is not None else i
    total_per_class = torch.stack(all_counts).sum(dim=0)
    total = total_per_class.sum()
    if return_ratio:
        ratio = total_per_class/total
    else:
        ratio = total_per_class
    return dict(zip(idx.tolist(), ratio))


In [ ]:
#| export
def interpolate_nan_clip(x_in, physiological_range_clip=None, percentile_clip=None, return_mask_only=False):
    """
    Function to clip outliers based on percentiles or physiological range and then interpolate nearby values
    """
    x = np.copy(x_in)
    if physiological_range_clip is not None:
        # if a physiological range, clip, set to nan, and interpolate nearby values
        assert len(physiological_range_clip) == 2, "physiological_range_clip expects a tuple or list of 1 or 2 values. Supply none to clip only one end."
        max_ = physiological_range_clip[1] if physiological_range_clip[1] is not None else None
        min_ = physiological_range_clip[0] if physiological_range_clip[0] is not None else None
        if max_ is not None:
            x[x>=max_] = np.nan
        if min_ is not None:
            x[x<=min_] = np.nan
    if percentile_clip is not None:
        # if percentiles, clip at percentiles, set to nana, and interpolate nearby
        assert len(percentile_clip) == 2, "percentile_clip expects a tuple or list of 1 or 2 values. Supply none to clip only one end."
        max_ = np.quantile(x, q=percentile_clip[1]) if percentile_clip[1] is not None else None
        min_ = np.quantile(x, q=percentile_clip[0]) if percentile_clip[0] is not None else None
        if max_ is not None:
            x[x>=max_] = np.nan
        if min_ is not None:
            x[x<=min_] = np.nan
    mask = np.isnan(x)
    if return_mask_only:
        return mask
    if all(mask):
        return x_in
    else:
        x[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), x[~mask])
        return x

In [ ]:
#| export
def calculate_stats(idx, zarr_file, channels, clip_interpolations=None, channel_magnitude_multiple=None):
    """
    Function to caluclate stats on an individual zarr array, including a clip interpolate range
    """
    root_grp = zarr.open(zarr_file)
    updated_channels = []
    if any(isinstance(i, list) for i in channels):
        avail_channels = list(root_grp.array_keys())
        for p in channels:
            updated_channels.append(next(x for x in p if x in avail_channels))
    else:
        updated_channels = channels
    channel_tracker = {channel:{'std':0,'len':0,'mean':0, 'n':0, 'n_zeros':0, 'n_nan':0, '50%':0, '95%':0, '99%':0, '5%':0, '1%':0} for channel in updated_channels}
    for channel in updated_channels:
        if channel in root_grp.array_keys():
            channel_tracker[channel]['n'] += 1
            temp = root_grp[channel][:].astype(np.float32)
            if clip_interpolations is not None and clip_interpolations != {} and channel in clip_interpolations:
                temp = interpolate_nan_clip(temp, physiological_range_clip=clip_interpolations[channel]['phys_range'], percentile_clip=clip_interpolations[channel]['percentiles'])
            channel_tracker[channel]['n_zeros'] = (temp == 0).sum()
            channel_tracker[channel]['n_nan'] = np.isnan(temp).sum()
            channel_tracker[channel]['file'] = zarr_file
            channel_tracker[channel]['mean'] += np.nanmean(temp)
            channel_tracker[channel]['len'] += len(temp)
            channel_tracker[channel]['std'] += np.nanstd(temp)
            quantiles = np.nanquantile(temp, q=[0.01,0.05,0.5,0.95,0.99])
            for idx, quantile in enumerate(['1%','5%','50%', '95%','99%']):
                channel_tracker[channel][quantile] += quantiles[idx]
    if channel_magnitude_multiple is not None:
        for channel in updated_channels:
            if channel in channel_magnitude_multiple:
                for stat in ['mean', 'std', '1%','5%','50%', '95%','99%']:
                    channel_tracker[channel][stat] *= channel_magnitude_multiple[channel]
    return channel_tracker

def calculate_stats_all(zarr_files, channels, sample_wise=True, clip_interpolations=None, channel_magnitude_multiple=None):
    with mp.Pool() as pool:
        f = partial(calculate_stats, channels=channels, clip_interpolations=clip_interpolations, channel_magnitude_multiple=channel_magnitude_multiple)
        results = pool.starmap_async(f, enumerate(zarr_files))
        pool.close()
        pool.join()
    if not sample_wise:
        all_channels = [v for i in channels for v in i]
        total_stats = {channel:{'std':0,'len':0,'mean':0, 'n':1, 'n_zeros':0, 'n_nan':0, '50%':0, '95%':0, '99%':0, '5%':0, '1%':0} for channel in all_channels}
        for result in results.get():
            for channel in result:
                for stat in ['n','len','n_zeros','n_nan', 'std','mean','1%','5%','50%','95%','99%']:
                    total_stats[channel][stat] += result[channel][stat]
        for channel in total_stats:
            for stat in ['std','mean','1%','5%','50%','95%','99%']:
                total_stats[channel][stat] = total_stats[channel][stat] / total_stats[channel]['n']
        return pd.DataFrame(total_stats).T
    else:
        final_df = pd.DataFrame()
        for result in results.get():
            final_df = pd.concat([final_df, pd.DataFrame(result).T])
        return final_df

In [ ]:
#| export
def calculate_samples(idx, zarr_file, channels, frequency, sample_seq_len_sec, stride_sec, start_offset_sec=None, max_seq_len_sec=None, include_partial_samples=True, nan_tolerance=0.0):
    """
    Function to create a dataframe of samples and their sequence indices
    """
    if start_offset_sec is None:
        start_offset_sec = 0
    start_offset = start_offset_sec * frequency
    if max_seq_len_sec is not None:
        assert max_seq_len_sec >= sample_seq_len_sec, "The maximum sequence length should be >= the sample sequence length. The maximum sequence length is the end cutoff point of a sample. A sample cannot be longer than that value."
    sample_seq_len = sample_seq_len_sec * frequency

    stride = stride_sec * frequency
    root_grp = zarr.open(zarr_file)
    updated_channels = []
    avail_channels = list(root_grp.array_keys())
    if all(isinstance(i, list) for i in channels):
        for p in channels:
            updated_channels.append(next((x for x in p if x in avail_channels), None))
    else:
        updated_channels = [p if p in avail_channels else None for p in channels]
    if None not in updated_channels:
        # all channels are present
        ## all channels should be the same length
        if 'header' in root_grp.attrs:
            duration = int(root_grp.attrs['header']['Duration']) # duration in seconds
        else:
            duration = int(root_grp.attrs['Duration']) # duration in seconds
        if duration > start_offset_sec:
            if include_partial_samples:
                max_seq_len = max_seq_len_sec*frequency if max_seq_len_sec is not None else duration*frequency+sample_seq_len-1
                sample_indices = [(i, i+sample_seq_len) for i in range(start_offset, max_seq_len+start_offset, stride) if (i-start_offset)+sample_seq_len <= max_seq_len]#duration*frequency+sample_seq_len]
            else:
                max_seq_len =  max_seq_len_sec*frequency if max_seq_len_sec is not None and max_seq_len_sec < duration else duration*frequency
                sample_indices = [(i, i+sample_seq_len) for i in range(start_offset, max_seq_len+start_offset, stride) if (i-start_offset)+sample_seq_len <= max_seq_len]
            sample_index_df = pd.DataFrame([{'start_idx':i[0], 'end_idx':i[1]} for i in sample_indices])
            sample_index_df['file'] = zarr_file
            sample_index_df['zarr_index'] = idx # assign unique uuid to each sample
            if not sample_index_df.empty:
                for channel in updated_channels:
                    channel_data = root_grp[channel][:]
                    sample_index_df[f'{channel}_sum_nan'] = sample_index_df.apply(
                        lambda row: np.isnan(channel_data[row['start_idx']:row['end_idx']]).sum(), axis=1
                    )
                
                # the maximum channel nan sum should be less than the nan tolerance
                sample_index_df['all_channels_nan_max'] = sample_index_df[[f'{channel}_sum_nan' for channel in updated_channels]].max(axis=1)
                cutoff = nan_tolerance * sample_seq_len 
                sample_index_df = sample_index_df[sample_index_df['all_channels_nan_max'] <= cutoff]
                            
                sample_index_df['n_samples'] = len(sample_indices)
                return sample_index_df

def calculate_samples_mp(zarr_files, channels, frequency, sample_seq_len_sec, stride_sec, start_offset_sec = None, max_seq_len_sec=None, include_partial_samples=True, nan_tolerance=0.0):
    """
    Multiprocessing function to generate samples
    """
    final_df = pd.DataFrame(columns=['file', 'start_idx','end_idx','n_samples'])
    total_samples = 0
    with mp.Pool() as pool:
        f = partial(calculate_samples, channels=channels, frequency=frequency, start_offset_sec=start_offset_sec, max_seq_len_sec=max_seq_len_sec, sample_seq_len_sec=sample_seq_len_sec, stride_sec=stride_sec, include_partial_samples=include_partial_samples, nan_tolerance=nan_tolerance)
        results = pool.starmap_async(f, enumerate(zarr_files))
        pool.close()
        pool.join()
    for result in results.get():
        final_df = pd.concat([final_df, result])
    final_df.reset_index(drop=True, inplace=True)
    total_samples = len(final_df)
    return final_df, total_samples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()